### Helper methods

### Config

In [1]:
from db_helper_methods import *

import pymysql
import pandas as pd
import numpy as np
import time

db_name = "indecies_playground"

### Create DB

In [2]:
execute_query("CREATE DATABASE IF NOT EXISTS `{db_name}`;")

print(f"Database '{db_name}' ensured to exist.")

Database 'indecies_playground' ensured to exist.


### Multi index table creation

In [3]:
table_name = "index_test"

# Drop table if exists
execute_query(query=f"DROP TABLE IF EXISTS {table_name}", database=db_name)

# Create table with composite PRIMARY KEY (employee_id, department_id)
create_table_sql = f"""
CREATE TABLE {table_name} (
    employee_id INT NOT NULL,
    department_id INT NOT NULL,
    name VARCHAR(100),
    PRIMARY KEY (employee_id, department_id)
);
"""

execute_query(query=create_table_sql, database=db_name)

# Indsæt nogle testdata
employees = [
    (1, 10, 'Thomas'),
    (2, 10, 'Maria'),
    (3, 20, 'Bent'),
    (4, 20, 'Sofie'),
    (1, 20, 'Anna')  # Samme employee_id, men anden department_id → tilladt
]

insert_sql = f"INSERT INTO {table_name} (employee_id, department_id, name) VALUES (%s, %s, %s)"

execute_many_query(query=insert_sql, params_list=employees, database=db_name)

In [4]:
import json
from pprint import pprint

table_name = "index_test"

query = f"EXPLAIN FORMAT=JSON SELECT * FROM {table_name} WHERE name = 'Thomas';"

cursor = execute_query(query, database=db_name)
result = cursor.fetchone()  # kun én række med JSON
plan_json = json.loads(result[0])  # result[0] = selve JSON-strengen

# Pretty print som struktur
pprint(plan_json)

{'query_block': {'cost_info': {'query_cost': '0.75'},
                 'select_id': 1,
                 'table': {'access_type': 'ALL',
                           'attached_condition': '(`indecies_playground`.`index_test`.`name` '
                                                 "= 'Thomas')",
                           'cost_info': {'data_read_per_join': '416',
                                         'eval_cost': '0.10',
                                         'prefix_cost': '0.75',
                                         'read_cost': '0.65'},
                           'filtered': '20.00',
                           'rows_examined_per_scan': 5,
                           'rows_produced_per_join': 1,
                           'table_name': 'index_test',
                           'used_columns': ['employee_id',
                                            'department_id',
                                            'name']}}}


In [7]:
drop_non_clustered_indexes(table_name, db_name)

execute_query(f"CREATE INDEX idx_name ON {table_name} (name);", database=db_name)

show_table_indexes(table_name, db_name)

Found 1 non-clustered indexes to drop
Dropped index 'idx_name'
Successfully dropped 1 of 1 indexes


,Index Name,Columns,Type,Non Unique
0,idx_name,name,BTREE,True
1,PRIMARY,"employee_id,department_id",BTREE,False
